In [1]:
%run moon_code
%run xgb_modelfit

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Data load
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)
# 사용할 12개 주식 목록 
stock_li = ['현대차', 'SK텔레콤', 'LG화학', '셀트리온', 
            'KB금융', '삼성전자', 'SK하이닉스', 'NAVER', 
            '현대글로비스', '현대건설', '삼성중공업', 'POSCO']
# 12개 주식만 남기기
key_li = list(raw_data.keys())
for i in key_li:
    if i not in stock_li:
        del raw_data[i]
# KOSPI data load
with open('ks11_20181008.pickle', 'rb') as handle:
    ks11 = pickle.load(handle)
# KOSPI 수익률 추가
for i in [1,3,5,7,10]:
    ks11['returnsClosePrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Close.shift(i))
    ks11['returnsOpenPrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Open.shift(i))
# KOSPI Volume==0인 index 저장
kospi_index_del = ks11[ks11.Volume == 0][ks11.columns[:6]].index
# KOSPI Volume ==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(ks11[ks11.Volume == 0][ks11.columns[:6]].index, axis=0)
# stock에서 Volume==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(df.index[np.where(df.Volume==0)[0]], axis=0)
# feature 생성
for key, df in raw_data.items():
    df["pct_change"] = df.Close.pct_change()
    for i in range(3, 31, 2):
        df["close_ma_{}".format(i)] = df.Close.rolling(window = i).mean()
        df["volume_ma_{}".format(i)] = df.Volume.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        df["his_vol_{}".format(i)] = df["pct_change"].rolling(window = i).std()*(252**0.5)
    for i in [1, 3, 5, 7, 10]:
        df['returnsClosePrevRaw{}'.format(i) ]= np.log(df.Open / df.Close.shift(i))
        df['returnsOpenPrevRaw{}'.format(i) ] = np.log(df.Open / df.Open.shift(i))
        df['returnsClosePrevMktres{}'.format(i)] = df['returnsClosePrevRaw{}'.format(i)] - ks11['returnsClosePrevRaw{}'.format(i)]
        df['returnsOpenPrevMktres{}'.format(i)] = df['returnsOpenPrevRaw{}'.format(i)] - ks11['returnsOpenPrevRaw{}'.format(i)]
    for i in [1, 3, 5, 7, 10]:
        df['Y3_{}'.format(i)] = np.where(df.Close.rolling(i).mean() - df.Close.shift(1).rolling(i).mean()>=0,1,0)
        df['sh1_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-1)
        df['sh5_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-5)
# 날짜 조정
data = adjust_window(raw_data, '2013-07-01', '2018-07-01')

li = ['Y3_{}'.format(i) for i in [1,3,5,7,10]]
li.extend(['sh1_Y3_{}'.format(i) for i in [1,3,5,7,10]])
li.extend(['sh5_Y3_{}'.format(i) for i in [1,3,5,7,10]])
target_li = li

X_li = [i for i in data['삼성전자'].columns if i not in target_li]
scl_X_li = ['scl_'+i for i in X_li]

for key, df in data.items():
    for X in X_li:
        df['scl_{}'.format(X)] = df[X]
        
train, test = multi_train_test_split(data, 0.8, dtrain=True)

multi_apply_standard_scale(train, test, scl_X_li)

scl_li = scl_X_li.copy()
for ix, i in enumerate(scl_X_li):
    if (i.find('returns') != -1) or (i.find('his') != -1) or (i.find('pct') != -1):
        scl_li[ix] = scl_X_li[ix][4:]
origin_X = [i for i in X_li if i.find('returns') != 0]
origin_scl_X = [i for i in scl_li if i.find('returns') != 0]

# 세부 내용 정리

In [3]:
# Data load
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)

In [4]:
len(raw_data)

202

In [5]:
raw_data['삼성전자'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,16060.0,16180.0,16000.0,16180.0,5.953252,11950800
2010-01-05,16520.0,16580.0,16300.0,16440.0,6.048919,27925850
2010-01-06,16580.0,16820.0,16520.0,16820.0,6.188735,22948850
2010-01-07,16820.0,16820.0,16260.0,16260.0,5.982689,22107950
2010-01-08,16400.0,16420.0,16120.0,16420.0,6.041558,14777550


In [6]:
# 사용할 12개 주식 목록 
stock_li = ['현대차', 'SK텔레콤', 'LG화학', '셀트리온', 
            'KB금융', '삼성전자', 'SK하이닉스', 'NAVER', 
            '현대글로비스', '현대건설', '삼성중공업', 'POSCO']
# 12개 주식만 남기기
key_li = list(raw_data.keys())
for i in key_li:
    if i not in stock_li:
        del raw_data[i]

In [7]:
len(raw_data)

12

In [8]:
set(raw_data.keys()) == set(stock_li)

True

In [9]:
# KOSPI data load
with open('ks11_20181008.pickle', 'rb') as handle:
    ks11 = pickle.load(handle)

In [10]:
ks11.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,1681.709961,1696.140015,1681.709961,1696.140015,1696.140015,295600
2010-01-05,1701.619995,1702.390015,1686.449951,1690.619995,1690.619995,407600
2010-01-06,1697.880005,1706.890015,1696.099976,1705.319946,1705.319946,425400
2010-01-07,1702.920044,1707.900024,1683.449951,1683.449951,1683.449951,461600
2010-01-08,1694.060059,1695.260010,1668.839966,1695.260010,1695.260010,379200


In [11]:
# KOSPI 수익률 추가
for i in [1,3,5,7,10]:
    ks11['returnsClosePrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Close.shift(i))
    ks11['returnsOpenPrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Open.shift(i))

In [12]:
ks11[['returnsClosePrevRaw3','returnsOpenPrevRaw3']].iloc[5:10]

,returnsClosePrevRaw3,returnsOpenPrevRaw3
Date,,
2010-01-11,-0.002660,0.001712
2010-01-12,0.007327,-0.004172
2010-01-13,-0.006955,-0.006247
2010-01-14,-0.007965,-0.011894
2010-01-15,-0.002352,-0.000696


In [13]:
ks11[['Close','Open']].iloc[:10]

,Close,Open
Date,,
2010-01-04,1696.140015,1681.709961
2010-01-05,1690.619995,1701.619995
2010-01-06,1705.319946,1697.880005
2010-01-07,1683.449951,1702.920044
2010-01-08,1695.260010,1694.060059
2010-01-11,1694.119995,1700.790039
2010-01-12,1698.640015,1695.829956
2010-01-13,1671.410034,1683.510010
2010-01-14,1685.770020,1680.680054


In [14]:
1702.920044 / 1681.709961

1.0126122122672019

In [15]:
(np.log(ks11.Open / ks11.Open.shift(3))).iloc[5:10]

Date
2010-01-11    0.001712
2010-01-12   -0.004172
2010-01-13   -0.006247
2010-01-14   -0.011894
2010-01-15   -0.000696
Name: Open, dtype: float64

In [16]:
# KOSPI Volume==0인 index 저장
kospi_index_del = ks11[ks11.Volume == 0][ks11.columns[:6]].index
kospi_index_del

DatetimeIndex(['2014-01-20', '2014-03-19', '2014-04-14', '2014-11-27',
               '2014-12-03', '2014-12-09', '2014-12-15', '2014-12-18',
               '2016-03-25', '2016-12-26', '2017-04-14', '2017-11-23'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [17]:
ks11.loc[kospi_index_del,'Volume']

Date
2014-01-20    0
2014-03-19    0
2014-04-14    0
2014-11-27    0
2014-12-03    0
2014-12-09    0
2014-12-15    0
2014-12-18    0
2016-03-25    0
2016-12-26    0
2017-04-14    0
2017-11-23    0
Name: Volume, dtype: int32

In [18]:
raw_data['삼성전자'].loc[kospi_index_del]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-20,26100.0,26380.0,25920.0,26320.0,71.011551,8682800
2014-03-19,25600.0,25700.0,25360.0,25380.0,68.475433,7638550
2014-04-14,27300.0,27720.0,27300.0,27400.0,73.925407,10586000
2014-11-27,25460.0,26020.0,25280.0,25280.0,69.517273,44377800
2014-12-03,26000.0,26020.0,25900.0,25960.0,71.387207,9735500
2014-12-09,26380.0,27140.0,26380.0,26980.0,74.192093,13553100
2014-12-15,25600.0,25600.0,25200.0,25460.0,70.012260,14337950
2014-12-18,25320.0,25580.0,25040.0,25320.0,69.627281,11808250
2016-03-25,25660.0,25800.0,25560.0,25760.0,1258.291992,7172300


In [19]:
# KOSPI Volume ==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(kospi_index_del, axis=0)

In [20]:
try:
    raw_data['삼성전자'].loc[kospi_index_del]
except KeyError:
    print('KeyError')

KeyError


In [21]:
# stock에서 Volume==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(df.index[np.where(df.Volume==0)[0]], axis=0)

In [22]:
for key, df in raw_data.items():
    print(len(df),'\t', key)
print(len(ks11), '\t', 'KOSPI')

2144 	 KB금융
2144 	 LG화학
2123 	 NAVER
2144 	 POSCO
2144 	 SK텔레콤
2144 	 SK하이닉스
2141 	 삼성전자
2144 	 삼성중공업
2130 	 셀트리온
2144 	 현대건설
2144 	 현대글로비스
2144 	 현대차
2151 	 KOSPI


In [23]:
# feature 생성
for key, df in raw_data.items():
    df["pct_change"] = df.Close.pct_change()
    for i in range(3, 31, 2):
        df["close_ma_{}".format(i)] = df.Close.rolling(window = i).mean()
        df["volume_ma_{}".format(i)] = df.Volume.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        df["his_vol_{}".format(i)] = df["pct_change"].rolling(window = i).std()*(252**0.5)
    for i in [1, 3, 5, 7, 10]:
        df['returnsClosePrevRaw{}'.format(i) ]= np.log(df.Open / df.Close.shift(i))
        df['returnsOpenPrevRaw{}'.format(i) ] = np.log(df.Open / df.Open.shift(i))
        df['returnsClosePrevMktres{}'.format(i)] = df['returnsClosePrevRaw{}'.format(i)] - ks11['returnsClosePrevRaw{}'.format(i)]
        df['returnsOpenPrevMktres{}'.format(i)] = df['returnsOpenPrevRaw{}'.format(i)] - ks11['returnsOpenPrevRaw{}'.format(i)]
    for i in [1, 3, 5, 7, 10]:
        df['Y3_{}'.format(i)] = np.where(df.Close.rolling(i).mean() - df.Close.shift(1).rolling(i).mean()>=0,1,0)
        df['sh1_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-1)
        df['sh5_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-5)

In [26]:
raw_data['삼성전자'].to_excel('잘됬는지 Test.xlsx')